In [1]:
import numpy as np
import math
from numpy.linalg import norm

In [2]:
roads = [
    ((565,79), (568,133)),
    ((568,133), (568,176)),
    ((568,176), (551,203)),
    ((551,203), (553,301)),
    ((553,301), (524,394)),
    ((524,394), (478,382)),
    ((478,382), (400,385)),
    ((400,385), (305,379)),
    ((305,379), (220,380)),
    ((220,380), (91,378)),
    ((91,378), (17,375)),
    ((17,375), (17,166)),
    ((17,166), (45,136)),
    ((45,136), (14,131)),
    ((14,131), (17,166)),
    ((45,136), (115,145)),
    ((115,145), (227,130)),
    ((227,130), (248,153)),
    ((248,153), (256,124)),
    ((256,124), (227,130)),
    ((256,124), (409,103)),
    ((409,103), (565,79)),
    ((248,153), (332,193)),
    ((332,193), (360,217)),
    ((360,217), (410,217)),
    ((410,217), (482,220)),
    ((482,220), (551,203)),
    ((410,217), (400,270)),
    ((400,270), (360,217)),
    ((400,270), (400,385)),
    ((524,394), (552,438)),
    ((552,438), (545,481)),
    ((545,481), (518,498)),
    ((552,438), (591,436)),
    ((591,436), (603,412)),
    ((603,412), (612,319)),
    ((612,319), (600,285)),
    ((600,285), (625,244)),
    ((625,244), (706,171)),
    ((706,171), (642,170)),
    ((642,170), (568,176)),
    ((706,171), (818,73)),
    ((818,73), (808,36)),
    ((808,36), (645,65)),
    ((645,65), (565,79)),
    ((808,36), (883,31)),
    ((883,31), (932,36)),
    ((932,36), (933,60)),
    ((933,60), (878,93)),
    ((878,93), (860,67)),
    ((860,67), (818,73)),
    ((878,93), (835,128)),
    ((835,128), (773,164)),
    ((773,164), (755,180)),
    ((755,180), (765,230)),
    ((765,230), (831,240)),
    ((831,240), (884,245)),
    ((884,245), (921,246)),
    ((921,246), (933,60)),
    ((921,246), (905,383)),
    ((905,383), (897,441)),
    ((897,441), (886,558)),
    ((886,558), (767,540)),
    ((767,540), (643,540)),
    ((643,540), (618,557)),
    ((618,557), (598,534)),
    ((598,534), (545,481)),
    ((618,557), (577,591)),
    ((577,591), (538,625)),
    ((538,625), (501,659)),
    ((501,659), (463,690)),
    ((463,690), (442,694)),
    ((442,694), (414,696)),
    ((414,696), (383,693)),
    ((383,693), (335,694)),
    ((335,694), (309,581)),
    ((309,581), (329,511)),
    ((329,511), (395,442)),
    ((395,442), (400,385)),
    ((395,442), (416,464)),
    ((416,464), (453,433)),
    ((453,433), (518,498)),
    ((518,498), (469,541)),
    ((469,541), (422,498)),
    ((422,498), (416,464)),
    ((469,541), (435,571)),
    ((435,571), (378,619)),
    ((378,619), (463,690)),
    ((383,693), (378,619)),
]

In [3]:
# const
HEIGHT = 1000
WIDTH = 1000

INF = 99999.999


In [4]:

class Point:
    def __init__(self, pos):
        self.pos = pos
        self.x, self.y = pos
        self.adjacents = [] # (road, adj_point)
        
    def _calc_dist(self, pos):
        return math.dist(self.pos, pos)
    
    def __eq__(self, _point: object):
        return self.x == _point.x and self.y == _point.y
    
    def __str__(self) -> str:
        return f'{self.pos}'

In [5]:
class Road:
    def __init__(self, from_point, to_point):
        self.from_point = from_point
        self.to_point = to_point
        self.from_pos = from_point.pos
        self.to_pos = to_point.pos
        self.length = math.dist(from_point.pos, to_point.pos)
        
        self.from_point.adjacents.append( (self, to_point) )
        self.to_point.adjacents.append( (self, from_point) )
    
    def _calc_dist(self, pos):
        p1 = np.asarray(self.from_pos)
        p2 = np.asarray(self.to_pos)
        p3 = np.asarray(pos)
        return np.abs(np.cross(p2-p1, p1-p3)) / norm(p2-p1)
    
    def _is_near(self, pos):
        x,y = pos
        x1,y1 = self.from_pos
        x2,y2 = self.to_pos
        
        is_x_middle = (x >= x1 and x <= x2) or (x <= x1 and x >= x2)
        is_y_middle = (y >= y1 and y <= y2) or (y <= y1 and y >= y2)
        
        return self._calc_dist(pos) <= 20 and (is_x_middle or is_y_middle)
    
    def __str__(self) -> str:
        return f'({self.from_pos}, {self.to_pos})'

In [6]:
class Dragger:
    def __init__(self) -> None:
        self.dragging = False

In [7]:
points = list(set([ x for x, y in roads]))
str_points = [ str(x) for x in points]
map_points = {}
for point in points:
    map_points[str(point)] = Point(point)
    
roads_lst = []
for from_pos, to_pos in roads:
    roads_lst.append(Road(map_points[str(from_pos)], map_points[str(to_pos)]))
    
for key in map_points:
    map_points[key].f = INF
    map_points[key].g = INF
    map_points[key].h = INF
    map_points[key].parent = None


In [8]:
start_pos = (14,131)
end_pos = (886, 558)

#Initialize
map_points[str(start_pos)].f = 0
map_points[str(start_pos)].g = 0
map_points[str(start_pos)].h = 0
map_points[str(start_pos)].parent = None

openlist = {str(start_pos)}
closedlist = set()
route_found = False

while len(openlist) > 0 and not route_found:
    #find the node with the least f on open list -> q
    minf = min([map_points[key].f for key in openlist])
    q = None
    for key in openlist:
        if minf == map_points[key].f:
            q = key
            break
    
    #pop q off open list
    openlist.remove(q)
    closedlist.add(q)
    
    #generate q's successors and set their parent to q
    point = map_points[q]
    for road, to_point in point.adjacents:
        
        #if successor is the goal, stop search
        current_f = to_point.f
        if to_point.pos == end_pos:
            to_point.parent = point
            print("FOUND")
            route_found = True
            break
            #trace path
            
            break
        elif not str(to_point) in closedlist:
            g_new = point.g + road.length
            h_new = point._calc_dist(end_pos)
            f_new = g_new + h_new
            
            if str(to_point) not in openlist or current_f > f_new:
                openlist.add(str(to_point))
                
                to_point.f = f_new
                to_point.g = g_new
                to_point.h = h_new
                to_point.parent = point

FOUND


In [9]:
node = map_points[str(end_pos)]
while node.parent is not None:
    print(node)
    node = node.parent
print(node.pos)

(886, 558)
(767, 540)
(643, 540)
(618, 557)
(598, 534)
(545, 481)
(552, 438)
(524, 394)
(553, 301)
(551, 203)
(482, 220)
(410, 217)
(360, 217)
(332, 193)
(248, 153)
(227, 130)
(115, 145)
(45, 136)
(14, 131)
